In [ ]:
# ============================================
# YOLOv8 Segmentation 完整訓練流程（修正版）
# 功能：自動處理 Roboflow COCO 格式並訓練水面檢測
# ============================================

import os
import json
import yaml
import shutil
import numpy as np
from pathlib import Path
from PIL import Image
import torch
from ultralytics import YOLO
import warnings
warnings.filterwarnings('ignore')

# ===== 1. 設定路徑 =====
BASE_DIR = Path(r"C:\Users\hanaj\Desktop\HanaJ\Lab\YOLOgroundtruth\Floating_Plants_water.v6i.coco-segmentation")
EXTRA_TEST_DIR = Path(r"C:\Users\hanaj\Desktop\HanaJ\Lab\cropped")

print("=" * 60)
print("🚀 YOLOv8 Segmentation 訓練程式")
print("=" * 60)
print(f"資料集路徑: {BASE_DIR}")
print(f"額外測試路徑: {EXTRA_TEST_DIR}")

# ===== 2. 重新組織資料夾結構（關鍵步驟！）=====
def reorganize_dataset(base_dir):
    """將 Roboflow 格式重組為 YOLO 格式"""
    print("\n📁 重新組織資料夾結構...")

    for split in ['train', 'valid', 'test']:
        split_dir = base_dir / split
        if not split_dir.exists():
            continue

        # 建立 images 子資料夾
        images_dir = split_dir / 'images'
        images_dir.mkdir(exist_ok=True)

        # 移動所有圖片到 images 資料夾
        for img_file in split_dir.glob('*.jpg'):
            dst = images_dir / img_file.name
            if not dst.exists():
                shutil.copy2(img_file, dst)

        for img_file in split_dir.glob('*.png'):
            dst = images_dir / img_file.name
            if not dst.exists():
                shutil.copy2(img_file, dst)

        print(f"✓ {split}/images: {len(list(images_dir.glob('*')))} 張圖片")

reorganize_dataset(BASE_DIR)

# ===== 3. COCO 轉 YOLO Segmentation（修正版）=====
def coco_to_yolo_segmentation(base_dir):
    """將 COCO polygon 轉換為 YOLO segmentation 格式"""
    print("\n🔄 轉換 COCO 到 YOLO 格式...")

    # 收集所有類別
    all_categories = set()

    for split in ['train', 'valid', 'test']:
        split_dir = base_dir / split
        coco_file = split_dir / '_annotations.coco.json'

        if not coco_file.exists():
            print(f"⚠️ 找不到 {coco_file}")
            continue

        with open(coco_file, 'r') as f:
            coco_data = json.load(f)

        # 收集類別
        for cat in coco_data.get('categories', []):
            all_categories.add(cat['name'])

    # 建立類別映射（固定順序）
    class_names = sorted(list(all_categories))
    class_to_id = {name: idx for idx, name in enumerate(class_names)}

    print(f"找到類別: {class_names}")

    # 對每個 split 進行轉換
    for split in ['train', 'valid', 'test']:
        split_dir = base_dir / split
        coco_file = split_dir / '_annotations.coco.json'
        labels_dir = split_dir / 'labels'

        if not coco_file.exists():
            continue

        # 建立 labels 資料夾
        labels_dir.mkdir(exist_ok=True)

        # 讀取 COCO 資料
        with open(coco_file, 'r') as f:
            coco_data = json.load(f)

        # 建立圖片 ID 到檔名的映射
        image_info = {}
        for img in coco_data['images']:
            image_info[img['id']] = {
                'file_name': img['file_name'],
                'width': img['width'],
                'height': img['height']
            }

        # 建立類別 ID 映射
        category_mapping = {}
        for cat in coco_data['categories']:
            category_mapping[cat['id']] = class_to_id[cat['name']]

        # 組織標註（按圖片分組）
        annotations_by_image = {}
        for ann in coco_data.get('annotations', []):
            img_id = ann['image_id']
            if img_id not in annotations_by_image:
                annotations_by_image[img_id] = []
            annotations_by_image[img_id].append(ann)

        # 轉換每張圖片的標註
        converted_count = 0
        for img_id, img_data in image_info.items():
            file_name = Path(img_data['file_name']).stem
            width = img_data['width']
            height = img_data['height']

            # 建立對應的標註檔案
            label_file = labels_dir / f"{file_name}.txt"

            # 取得這張圖片的所有標註
            annotations = annotations_by_image.get(img_id, [])

            lines = []
            for ann in annotations:
                if 'segmentation' not in ann:
                    continue

                # 取得類別 ID
                class_id = category_mapping.get(ann['category_id'])
                if class_id is None:
                    continue

                # 處理 segmentation（polygon）
                segmentation = ann['segmentation']
                if not segmentation or not isinstance(segmentation, list):
                    continue

                # 取第一個 polygon（COCO 可能有多個）
                if len(segmentation) > 0 and isinstance(segmentation[0], list):
                    polygon = segmentation[0]

                    # 確保是成對的 x,y 座標
                    if len(polygon) % 2 != 0:
                        polygon = polygon[:-1]

                    # 正規化座標
                    normalized_poly = []
                    for i in range(0, len(polygon), 2):
                        x = polygon[i] / width
                        y = polygon[i + 1] / height
                        # 確保座標在 0-1 範圍內
                        x = max(0, min(1, x))
                        y = max(0, min(1, y))
                        normalized_poly.extend([x, y])

                    # 確保至少有 3 個點（6 個座標值）
                    if len(normalized_poly) >= 6:
                        # 格式：class_id x1 y1 x2 y2 ...
                        line = f"{class_id} " + " ".join([f"{v:.6f}" for v in normalized_poly])
                        lines.append(line)

            # 寫入標註檔案
            with open(label_file, 'w') as f:
                if lines:
                    f.write('\n'.join(lines))
                    converted_count += 1
                else:
                    # 如果沒有標註，建立空檔案
                    f.write('')

        print(f"✓ {split}: 轉換了 {converted_count} 個標註檔案")

    return class_names

# 執行轉換
class_names = coco_to_yolo_segmentation(BASE_DIR)

# ===== 4. 建立 data.yaml =====
def create_data_yaml(base_dir, class_names):
    """建立 YOLO 所需的 data.yaml"""
    print("\n📝 建立 data.yaml...")

    data_yaml = {
        'path': str(base_dir),
        'train': 'train/images',
        'val': 'valid/images',
        'test': 'test/images',
        'names': class_names,
        'nc': len(class_names)
    }

    yaml_path = base_dir / 'data.yaml'
    with open(yaml_path, 'w') as f:
        yaml.dump(data_yaml, f, default_flow_style=False, sort_keys=False)

    print(f"✓ 已建立 {yaml_path}")
    print(f"  類別數量: {len(class_names)}")
    print(f"  類別名稱: {class_names}")

    return yaml_path

data_yaml_path = create_data_yaml(BASE_DIR, class_names)

# ===== 5. 驗證資料集 =====
def verify_dataset(base_dir):
    """驗證資料集是否正確設置"""
    print("\n🔍 驗證資料集...")

    for split in ['train', 'valid', 'test']:
        split_dir = base_dir / split
        images_dir = split_dir / 'images'
        labels_dir = split_dir / 'labels'

        if images_dir.exists() and labels_dir.exists():
            image_files = list(images_dir.glob('*.jpg')) + list(images_dir.glob('*.png'))
            label_files = list(labels_dir.glob('*.txt'))

            # 檢查非空標註檔案
            non_empty_labels = 0
            for label_file in label_files:
                if label_file.stat().st_size > 0:
                    non_empty_labels += 1

            print(f"  {split}:")
            print(f"    - 圖片數: {len(image_files)}")
            print(f"    - 標註數: {len(label_files)}")
            print(f"    - 非空標註: {non_empty_labels}")

            # 讀取一個範例標註來驗證格式
            if label_files and non_empty_labels > 0:
                for label_file in label_files[:5]:
                    if label_file.stat().st_size > 0:
                        with open(label_file, 'r') as f:
                            first_line = f.readline().strip()
                            if first_line:
                                parts = first_line.split()
                                if len(parts) > 6:  # class_id + 至少3個點(6個座標)
                                    print(f"    ✓ 範例標註格式正確 (點數: {(len(parts)-1)//2})")
                                    break

verify_dataset(BASE_DIR)

# ===== 6. 檢查 GPU =====
print("\n🖥️ 系統資訊:")
print(f"  PyTorch 版本: {torch.__version__}")
print(f"  CUDA 可用: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"  GPU: {torch.cuda.get_device_name(0)}")
    print(f"  顯存: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

# ===== 7. 訓練模型 =====
def train_model(data_yaml_path, base_dir):
    """訓練 YOLOv8 segmentation 模型"""
    print("\n🎯 開始訓練模型...")

    # 訓練參數（針對 RTX 2060 優化）
    model = YOLO('yolov8n-seg.pt')  # 使用較小的模型以確保成功

    # 訓練
    results = model.train(
        data=str(data_yaml_path),
        epochs=100,  # 先用較少的 epochs 測試
        imgsz=640,   # 標準尺寸
        batch=8,     # RTX 2060 適合的 batch size
        device=0 if torch.cuda.is_available() else 'cpu',
        project=str(base_dir / 'runs'),
        name='water_segmentation',
        exist_ok=True,
        patience=20,
        save=True,
        save_period=10,
        plots=True,
        verbose=True,
        seed=42,
        deterministic=True,
        workers=4,
        amp=True,  # 混合精度訓練
        val=True,
        # 資料增強參數（針對水面檢測優化）
        hsv_h=0.015,
        hsv_s=0.7,
        hsv_v=0.4,
        degrees=0.0,
        translate=0.1,
        scale=0.5,
        shear=0.0,
        perspective=0.0,
        flipud=0.0,
        fliplr=0.5,
        mosaic=1.0,
        mixup=0.0,
        copy_paste=0.0
    )

    print("✓ 訓練完成！")
    return results

# 執行訓練
try:
    results = train_model(data_yaml_path, BASE_DIR)

    # ===== 8. 測試模型 =====
    print("\n🧪 測試模型...")

    # 載入最佳權重
    best_model_path = BASE_DIR / 'runs' / 'water_segmentation' / 'weights' / 'best.pt'
    if not best_model_path.exists():
        best_model_path = BASE_DIR / 'runs' / 'water_segmentation' / 'weights' / 'last.pt'

    if best_model_path.exists():
        model = YOLO(best_model_path)

        # 在測試集上評估
        test_dir = BASE_DIR / 'test' / 'images'
        if test_dir.exists():
            test_images = list(test_dir.glob('*.jpg')) + list(test_dir.glob('*.png'))
            if test_images:
                print(f"\n測試 {len(test_images)} 張圖片...")

                for img_path in test_images[:3]:  # 測試前3張
                    results = model(img_path, conf=0.25, iou=0.45)

                    # 顯示結果
                    for r in results:
                        if r.masks is not None:
                            print(f"  ✓ {img_path.name}: 檢測到 {len(r.masks)} 個物件")
                        else:
                            print(f"  ✗ {img_path.name}: 未檢測到物件")

                    # 儲存結果
                    output_dir = BASE_DIR / 'test_results'
                    output_dir.mkdir(exist_ok=True)
                    results[0].save(str(output_dir / f"result_{img_path.name}"))

        # 測試額外的圖片
        if EXTRA_TEST_DIR.exists():
            extra_images = list(EXTRA_TEST_DIR.glob('*.jpg')) + list(EXTRA_TEST_DIR.glob('*.png'))
            if extra_images:
                print(f"\n測試額外圖片 ({len(extra_images)} 張)...")
                for img_path in extra_images[:3]:
                    results = model(img_path, conf=0.25, iou=0.45)
                    print(f"  處理: {img_path.name}")

    print("\n✅ 所有步驟完成！")
    print(f"📊 訓練結果保存在: {BASE_DIR / 'runs' / 'water_segmentation'}")
    print(f"📈 查看 results.png 和 confusion_matrix.png 以了解訓練效果")

except Exception as e:
    print(f"\n❌ 訓練失敗: {e}")
    print("可能的解決方案：")
    print("1. 檢查標註檔案是否正確生成")
    print("2. 降低 batch size 或 imgsz")
    print("3. 確認 GPU 驅動程式是否最新")


🚀 YOLOv8 Segmentation 訓練程式
資料集路徑: C:\Users\hanaj\Desktop\HanaJ\Lab\YOLOgroundtruth\Floating_Plants_water.v6i.coco-segmentation
額外測試路徑: C:\Users\hanaj\Desktop\HanaJ\Lab\cropped

📁 重新組織資料夾結構...
✓ train/images: 294 張圖片
✓ valid/images: 19 張圖片
✓ test/images: 16 張圖片

🔄 轉換 COCO 到 YOLO 格式...
找到類別: ['water-plants']
✓ train: 轉換了 294 個標註檔案
✓ valid: 轉換了 19 個標註檔案
✓ test: 轉換了 16 個標註檔案

📝 建立 data.yaml...
✓ 已建立 C:\Users\hanaj\Desktop\HanaJ\Lab\YOLOgroundtruth\Floating_Plants_water.v6i.coco-segmentation\data.yaml
  類別數量: 1
  類別名稱: ['water-plants']

🔍 驗證資料集...
  train:
    - 圖片數: 294
    - 標註數: 294
    - 非空標註: 294
    ✓ 範例標註格式正確 (點數: 34)
  valid:
    - 圖片數: 19
    - 標註數: 19
    - 非空標註: 19
    ✓ 範例標註格式正確 (點數: 142)
  test:
    - 圖片數: 16
    - 標註數: 16
    - 非空標註: 16
    ✓ 範例標註格式正確 (點數: 26)

🖥️ 系統資訊:
  PyTorch 版本: 2.4.1+cu121
  CUDA 可用: True
  GPU: NVIDIA GeForce RTX 2060
  顯存: 6.0 GB

🎯 開始訓練模型...
Ultralytics 8.3.203  Python-3.12.4 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2060, 6144MiB)
engine\trainer: 